**ALGORITHM 1: VL pretrained-M fine tuned with semi-disentangled outputs**

In [1]:
import requests
import os
import random
import json
import torch
import os
import kagglehub
import pickle
import math
import cv2

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from collections import Counter
from sklearn.decomposition import PCA
from skimage.morphology import skeletonize
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split


from torchvision import transforms as A

In [89]:
# path = kagglehub.dataset_download("albertozorzetto/carla-densely-annotated-driving-dataset")
# print("Path to dataset files:", path)

Dataset Creation (IMAGE form to EMBEDDING form)

In [90]:
# # Download the CARLA dataset using the Kaggle API
# dataset_name = "albertozorzetto/carla-densely-annotated-driving-dataset"
# destination_path = "/content/datasets"

# import os
# # Install Kaggle API
# !pip install kaggle --upgrade
# # Upload kaggle.json for authentication
# from google.colab import files
# uploaded = files.upload()
# # Move kaggle.json to the proper directory
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# # Verify Kaggle is set up correctly
# !kaggle datasets list
# !kaggle datasets download -d {dataset_name} -p {destination_path} --unzip
# print("Path to dataset files:", destination_path)

# import tarfile
# images_path = os.path.join(destination_path, "images")
# labels_path = os.path.join(destination_path, "labels")
# # Create directories for extracted files
# os.makedirs(images_path, exist_ok=True)
# os.makedirs(labels_path, exist_ok=True)
# # Extract and manage .tar files
# tar_files = [f for f in os.listdir(destination_path) if f.endswith('.tar')]
# for tar_file in tar_files:
#     tar_path = os.path.join(destination_path, tar_file)
#     print(f"Extracting {tar_path}...")
#     with tarfile.open(tar_path) as tar:
#         # Determine the folder to extract based on the tar file name
#         if "images" in tar_file.lower():
#             tar.extractall(path=images_path)
#         elif "labels" in tar_file.lower():
#             tar.extractall(path=labels_path)
#         else:
#             print(f"Skipping unknown .tar file: {tar_file}")
#     os.remove(tar_path)  # Optional: Remove the .tar file after extraction
# print("All .tar files have been extracted.")
# print(f"Images folder: {images_path}")
# print(f"Labels folder: {labels_path}")

In [126]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

root = "datasets"
images_dir = f"{root}/images"
labels_dir = f"{root}/labels"
waypoint_file = "dataset_waypoints.json"
batch_size = 2

skip = 50

*Saving the {way points} for passing to the LLM*

In [130]:
def generate_waypoints(label, num_waypoints):
    if len(label.shape) == 3:
        label = cv2.cvtColor(label, cv2.COLOR_RGB2GRAY)
    
    road_mask = (label == 90).astype(np.uint8)
    
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return []
    
    largest_contour = max(contours, key=cv2.contourArea)
    binary_image = (road_mask > 0).astype(np.uint8)
    skeleton = skeletonize(binary_image).astype(np.uint8)
    
    y_coords, x_coords = np.where(skeleton > 0)
    path_points = np.column_stack((x_coords, y_coords))
    
    x_min, x_max = path_points[:, 0].min(), path_points[:, 0].max()
    y_min, y_max = path_points[:, 1].min(), path_points[:, 1].max()

    x_range = (x_max - x_min) * 0.25
    y_range = (y_max - y_min) * 0.25

    path_points = path_points[
        (path_points[:, 0] > x_min + x_range) & (path_points[:, 0] < x_max - x_range) &
        (path_points[:, 1] > y_min + y_range) & (path_points[:, 1] < y_max - y_range)
    ]
    
    if len(path_points) == 0:
        return []


    sampled_indices = np.linspace(0, len(path_points) - 1, num_waypoints, dtype=int)
    waypoints = path_points[sampled_indices]
    
    # print("Min/Max path points:", path_points.min(), path_points.max())

    # plt.imshow(skeleton, cmap="gray")
    # plt.scatter(x_coords, y_coords, c="red", marker="o")
    # plt.show()

    # print(waypoints)
    # print("x", waypoints[:,0])
    # print("y", waypoints[:,1])

    # num_selected_waypoints = min(25, len(path_points))
    # sampled_indices = np.linspace(0, len(path_points) - 1, num_selected_waypoints, dtype=int)
    # waypoints = path_points[sampled_indices]

    # if len(waypoints) > 0:
    #     plt.figure(figsize=(10, 10))
    #     plt.imshow(label, cmap="gray")
    #     plt.scatter(waypoints[:, 0], waypoints[:, 1], c='red', marker='o')
    #     for i, (x, y) in enumerate(waypoints):
    #         plt.text(x, y, f'({x:.1f}, {y:.1f})', fontsize=8, color='yellow', ha='right', va='bottom')
    #     plt.title("Extracted Waypoints")
    #     plt.show()

    return waypoints.tolist()


In [131]:
def process_dataset(labels_dir, output_dir, num_waypoints):
    # os.makedirs(output_dir, exist_ok=True)
    dataset_waypoints = []

    for i in range(0, 28):
        k = "0" if i <= 9 else ""
        curr_folder_labels = f"{labels_dir}/Video_0{k}{i}"
        labels_path = [file for file in os.listdir(curr_folder_labels) if file.endswith(".png")]
        labels_path.sort()
        labels_path = labels_path[::skip]

        for label_file in labels_path:
            label_path = os.path.join(curr_folder_labels, label_file)
            label = Image.open(label_path)
            label_np = np.array(label)
            waypoints = generate_waypoints(label_np, num_waypoints=num_waypoints)
            output_label_path = os.path.join(output_dir, label_file)
            dataset_waypoints.append({
                "label": output_label_path,
                "waypoints": waypoints
            })
            # break

        with open(os.path.join(output_dir, waypoint_file), 'a') as wf:
            json.dump(dataset_waypoints, wf)
            wf.write("\n")
            # break

        if(i<27):
            print(f"\rProgress: {((i + 1) / 0.27):.2f} %", end = "")
        # break

    print(f"\nDone")

In [ ]:
process_dataset(labels_dir, root, 1000)

In [133]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, labels_dir):
        self.data = []
        ei = []
        cw = []

        for i in range(0, 28):
            k = "0" if i <= 9 else ""
            curr_folder_images = f"{images_dir}/Video_0{k}{i}"
            images_path = [file for file in os.listdir(curr_folder_images) if file.endswith(".png")]
            images_path.sort()
            images_path = images_path[::skip]

            for image_file in images_path:
                image_path = os.path.join(curr_folder_images, image_file)
                image = Image.open(image_path)
                image_embeddings = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)
                ei.append(image_embeddings['pixel_values'])

            with open(os.path.join(root, waypoint_file), "r") as wf:
                content = wf.read()
                data_list = content.splitlines()
                for data in data_list:
                    data = json.loads(data)
                    for i in data:
                        corresponding_waypoints = i['waypoints']
                        cw.append(corresponding_waypoints)
                        # print(corresponding_waypoints)
                        # break
                    # break
            for e,c in zip(ei, cw):
                self.data.append((e,c))
                # break
            # break
        # print(self.data[0][1])
        # print(self.data[0][0])
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        embedding_inputs, embedding_labels = self.data[idx]
        return embedding_inputs, embedding_labels

In [29]:
# Split indices for train and validation
dataset = SegmentationDataset(images_dir, labels_dir)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, pin_memory=True)

train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

val_dataset = torch.utils.data.Subset(dataset, val_indices)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [9]:
def normalize(pixel_values):
    mini = pixel_values.min()
    maxi = pixel_values.max()
    pixel_values = (pixel_values - mini) / (maxi - mini)
    return pixel_values

OUR MODEL

In [10]:
class WaypointModel(nn.Module):
    def __init__(self):
        super(WaypointModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 50)

        self.dropout = nn.Dropout(0.25)

    def forward(self, pixel_values):
        x = torch.relu(self.conv1(pixel_values))
        x = self.bn1(x)
        x = torch.max_pool2d(x, 2)
        
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv3(x))
        x = self.bn3(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv4(x))
        x = self.bn4(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv5(x))
        x = self.bn5(x)
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        output = self.fc2(x)
        return output.view(-1, 25, 2).squeeze()

In [11]:
def to_float(val):
    return val.float()

In [12]:
def model_output_generation(pixel_values, vit_wp_model):
    pixel_values = pixel_values.squeeze(1)
    outputs = vit_wp_model(pixel_values)
    return outputs

*--training--*

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# vit_wp_model = vit_wp_model.to(device)

In [30]:
vit_wp_model = WaypointModel()

In [31]:
criterion = nn.MSELoss()
optimizer = optim.Adam(vit_wp_model.parameters(), lr=1e-2)

In [32]:
def compute_accuracy(outputs, targets):
  threshold = targets.mean() * 0.01
  # print("outputs:", outputs)
  # print("targets:", targets)
  MSE = torch.sqrt(torch.sum((outputs - targets) ** 2, dim=-1))
  accurate_predictions = (MSE <= threshold).float()
  accuracy = accurate_predictions.mean().item()
  # print("MSE:", MSE)
  # print("acc preds:", accurate_predictions)
  # print("acc:", accuracy)
  return accuracy

In [33]:
def wayp_to_targets(way_p):
    # print(way_p)
    return to_float(torch.stack([torch.stack(pair, dim = 0) for pair in way_p]).permute(2,0,1))

In [34]:
def test(vit_wp_model, dataloader, optimizer, criterion):
    vit_wp_model.eval()
    running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, (p_val, way_p) in enumerate(dataloader):
            print("way_p len", len(way_p))
            print("wayp:", way_p)
            target = wayp_to_targets(way_p)[0]
            target = target.view(25, 2)
            optimizer.zero_grad()
            print("target shape", target.shape)
            print("target:", target)

            

            outputs = model_output_generation(p_val[0], vit_wp_model)
            print("output shape", outputs.shape)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            print(way_p)
            print(outputs)
            # break
            
    return running_loss / len(dataloader)

In [46]:
test_loss = test(vit_wp_model, dataloader, optimizer, criterion)
print(f"test loss is: {test_loss}")

way_p len 10
wayp: [[tensor([384, 403]), tensor([304, 298])], [tensor([357, 262]), tensor([322, 320])], [tensor([277, 242]), tensor([333, 332])], [tensor([328, 220]), tensor([342, 344])], [tensor([577,  67]), tensor([353, 354])], [tensor([257, 127]), tensor([368, 365])], [tensor([94, 85]), tensor([398, 381])], [tensor([269,  18]), tensor([444, 417])], [tensor([583, 118]), tensor([502, 469])], [tensor([163, 136]), tensor([599, 598])]]
target shape torch.Size([10, 2])
target: tensor([[384., 304.],
        [357., 322.],
        [277., 333.],
        [328., 342.],
        [577., 353.],
        [257., 368.],
        [ 94., 398.],
        [269., 444.],
        [583., 502.],
        [163., 599.]])
output shape torch.Size([10, 2])
[[tensor([384, 403]), tensor([304, 298])], [tensor([357, 262]), tensor([322, 320])], [tensor([277, 242]), tensor([333, 332])], [tensor([328, 220]), tensor([342, 344])], [tensor([577,  67]), tensor([353, 354])], [tensor([257, 127]), tensor([368, 365])], [tensor([94, 8

way_p len 10
wayp: [[tensor([451, 451]), tensor([316, 316])], [tensor([261, 261]), tensor([339, 339])], [tensor([208, 208]), tensor([352, 352])], [tensor([173, 173]), tensor([366, 366])], [tensor([107, 107]), tensor([382, 382])], [tensor([38, 38]), tensor([401, 401])], [tensor([169, 169]), tensor([459, 459])], [tensor([47, 47]), tensor([508, 508])], [tensor([648, 648]), tensor([586, 586])], [tensor([31, 31]), tensor([592, 592])]]
target shape torch.Size([10, 2])
target: tensor([[451., 316.],
        [261., 339.],
        [208., 352.],
        [173., 366.],
        [107., 382.],
        [ 38., 401.],
        [169., 459.],
        [ 47., 508.],
        [648., 586.],
        [ 31., 592.]])
output shape torch.Size([10, 2])
[[tensor([451, 451]), tensor([316, 316])], [tensor([261, 261]), tensor([339, 339])], [tensor([208, 208]), tensor([352, 352])], [tensor([173, 173]), tensor([366, 366])], [tensor([107, 107]), tensor([382, 382])], [tensor([38, 38]), tensor([401, 401])], [tensor([169, 169]), tensor([459, 459])], [tensor([47, 47]), tensor([508, 508])], [tensor([648, 648]), tensor([586, 586])], [tensor([31, 31]), tensor([592, 592])]]
tensor([[-0.0217,  0.0139],
        [ 0.0400,  0.0036],
        [ 0.0327,  0.0203],
        [ 0.0192, -0.0134],
        [ 0.0304,  0.0422],
        [ 0.0328,  0.0433],
        [ 0.0071,  0.0051],
        [-0.0380, -0.0139],
        [-0.0003, -0.0257],
        [-0.0057, -0.0179]])
test loss is: 162.59809027777777

In [36]:
def training(vit_wp_model, dataloader, optimizer, criterion):
    vit_wp_model.train()
    running_loss = 0.0
    running_accuracy = 0.0

    for i, (p_val, way_p) in enumerate(dataloader):
        target = wayp_to_targets(way_p)[0]
        target = target.view(25, 2)
        optimizer.zero_grad()

        outputs = model_output_generation(p_val[0], vit_wp_model)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        accuracy = compute_accuracy(outputs, target)
        running_accuracy += accuracy
        
        loss.backward()
        optimizer.step()
        # break

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [37]:
def validation(vit_wp_model, dataloader, optimizer, criterion):
    vit_wp_model.eval()
    running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, (p_val, way_p) in enumerate(dataloader):
            target = wayp_to_targets(way_p)[0]
            target = target.view(25, 2)

            outputs = model_output_generation(p_val[0], vit_wp_model)
            loss = criterion(outputs, target)
            running_loss += loss.item()

            accuracy = compute_accuracy(outputs, target)
            running_accuracy += accuracy
            # break

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [43]:
vit_wp_model = WaypointModel()
criterion = nn.MSELoss()
num_epochs = 5
lr = 1e-1

In [44]:
best_vit_wp_model = None
for i in range(0,1):
  optimizer = optim.Adam(vit_wp_model.parameters(), lr=lr)
  best_train_loss = float('inf')
  best_val_loss = float('inf')
  # patience = 5
  print(f"Setting Learning rate to: {lr}")

  for epoch in range(num_epochs):
    lr = lr / 10
    best_vit_wp_model = vit_wp_model
    # if(patience == 0):
    #   break
    
    train_loss, train_accuracy = training(vit_wp_model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = validation(vit_wp_model, val_loader, optimizer, criterion)
    
    if(train_loss > (best_train_loss * 0.95) or (val_loss > (best_val_loss * 0.95))):
      vit_wp_model = best_vit_wp_model
      # patience -= 1
    else:
      best_train_loss = train_loss
      best_val_loss = val_loss
      best_vit_wp_model = vit_wp_model
      # patience = 5
      # torch.save(vit_wp_model.state_dict(), '/content/drive/MyDrive/my_model.pth')
    print(f"\rEpoch {epoch + 1}/{num_epochs}, train-Loss: {train_loss:.1f}, val-Loss: {val_loss:.1f}     ", end = "")#, patience: {patience}", end = "")
    # break
  print("\n")
  # break

Enhancing Learning rate at: 0.1
Epoch 5/5, train-Loss: 71731.1, val-Loss: 317790.9     4398.5     



In [70]:
driver_model = best_vit_wp_model

In [45]:
torch.save(best_vit_wp_model.state_dict(), 'best_vit_wp_model_updated.pth')

**CarLA simulation**

redundent code

In [47]:
import requests
import os
import random
import json
import torch
import os
import kagglehub
import pickle
import math
import cv2

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from collections import Counter
from sklearn.decomposition import PCA
from skimage.morphology import skeletonize
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split

from torchvision import transforms as A

In [2]:
# prev one
class WaypointModel(nn.Module):
    def __init__(self):
        super(WaypointModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 64, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 40)

        self.dropout = nn.Dropout(0.25)

    def forward(self, pixel_values):
        x = torch.relu(self.conv1(pixel_values))
        x = self.bn1(x)
        x = torch.max_pool2d(x, 2)
        
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv3(x))
        x = self.bn3(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv4(x))
        x = self.bn4(x)
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv5(x))
        x = self.bn5(x)
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        output = self.fc2(x)
        return output.view(-1, 10, 2)

In [48]:
def to_float(val):
    return val.float()

In [49]:
def model_output_generation(pixel_values, vit_wp_model):
    pixel_values = pixel_values.squeeze(1)
    outputs = vit_wp_model(pixel_values)
    return outputs

In [50]:
import carla
import random
import time
import kagglehub
import os
import torch
import requests
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import SegformerForSemanticSegmentation, SegformerConfig
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [6]:
driver_model = WaypointModel()
driver_model.load_state_dict(torch.load('best_vit_wp_model.pth', map_location=torch.device('cpu')))

C:\Users\devas\AppData\Local\Temp\ipykernel_2060\334242544.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  driver_model.load_state_dict(torch.load('best_model.pth', map_

<All keys matched successfully>

In [51]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

In [52]:
client = carla.Client('localhost', 2000)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

netstat -an | findstr 2000
ping localhost

In [53]:
def remove_all_actors():
    actors = world.get_actors()
    for actor in actors:
        if 'vehicle' in actor.type_id:
            actor.destroy()
    world.tick()
    print("previous ones destroyed")

In [54]:
remove_all_actors()

previous ones destroyed


In [55]:
ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

In [56]:
def EGO_relative_view(EGO):
    view = EGO.get_transform().location + carla.Location(x=0, y=0, z=10)
    return view

In [57]:
def set_cam_to_EGO(EGO, angle = 65):
    spectator = world.get_spectator()
    vehicle_transform = EGO.get_transform()
    spectator.set_transform(
        carla.Transform(
            EGO_relative_view(EGO),
            carla.Rotation(pitch = -angle, yaw = vehicle_transform.rotation.yaw, roll = 0)
        )
    )

*functions to control*

In [80]:
def steering(x, intended_x, s_power):
    towards = intended_x - x
    steer = towards * 0.0002
    steer = steer * (1 + s_power) / 1.5
    steer = max(-1, min(steer, 1))
    return steer

In [76]:
def TandB(y_mid, intended_y_mid, curr_y = 25):
    if(intended_y_mid <= curr_y):
        return 0, 1
    intended_y_mid = intended_y_mid if intended_y_mid < y_mid else y_mid
    return max(0.3, min(intended_y_mid / (2 * y_mid), 1)), 0

In [60]:
def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    return Image.fromarray(array[:, :, :3])

In [61]:
def image_to_emb(image):
    global emb, image_1
    image_1 = image
    image = process_image(image)
    emb = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)

In [62]:
def normalize_wp(way_p_1, top_w,top_h):
    way_p = way_p_1

    mini_f = float('inf')
    maxi_f = -float('inf')
    for i in way_p[0]:
        if(maxi_f < i[0]):
            maxi_f = i[0]
        if(mini_f > i[0]):
            mini_f = i[0]
        
    mini_s = float('inf')
    maxi_s = -float('inf')
    for i in way_p[1]:
        if(maxi_s < i[1]):
            maxi_s = i[1]
        if(mini_s > i[1]):
            mini_s = i[1]
        
    for i in way_p:
        i[0] = (i[0] - mini_f) / (maxi_f - mini_f)
        i[1] = (i[1] - mini_s) / (maxi_s - mini_s)
        i[0] *= top_h
        i[1] *= top_w

    return way_p

In [63]:
set_cam_to_EGO(ego)

calculation of metrics

In [64]:
RC = 0
IS = 0
DS = 0

*--route completion--*

In [65]:
def evaluate_RC(intended_x_mid, ego_x_position, intended_y_mid, ego_y_position):
    global rcc
    if intended_y_mid > ego_y_position:
        dist = math.sqrt(((intended_x_mid - ego_x_position) ** 2) + ((intended_y_mid - ego_y_position) ** 2))
        if dist <= 900:
            rcc = rcc + 1

*--infraction score--*

In [66]:
class CollisionSensor(object):
    def __init__(self, vehicle):
        self.vehicle = vehicle
        self.world = vehicle.get_world()
        self.collision_history = []

        blueprint = self.world.get_blueprint_library().find('sensor.other.collision')
        self.sensor = self.world.spawn_actor(blueprint, carla.Transform(), attach_to=self.vehicle)
        self.sensor.listen(self._on_collision)

    def _on_collision(self, event):
        self.collision_history.append(event)
        
    def get_collision_count(self):
        return len(self.collision_history)

In [67]:
def detect_red_light_violation():
    global red_light_violations
    ego_position = ego.get_location()
    traffic_lights = world.get_traffic_lights()
    red_light_violation = False
    for traffic_light in traffic_lights:
        light_position = traffic_light.get_location()
        distance_to_light = ego_position.distance(light_position)
        if distance_to_light < 20.0:
            if traffic_light.get_state() == carla.TrafficLightState.Red:
                red_light_violation = True

In [68]:
def evaluate_IS(speed_limit, red_light_violations, collision_sensor):
    global isc

    velocity = ego.get_velocity()
    speed = (velocity.x**2 + velocity.y**2 + velocity.z**2) ** 0.5 * 3.6
    
    if speed > speed_limit:
        isc -= 1
        
    if red_light_violations:
        isc -= 1

    if collision_sensor.get_collision_count() > 0:
        isc -= 10

    if isc < 0:
        isc = 0

*--driving score--*

In [69]:
def evaluate_DS():
    global RC
    global IS
    global dsc
    dsc = (RC + IS) / 2

*controlling based on vit_wp_model*

In [32]:
print("started")

emb = None
prev_emb = None
image_1 = None

RC = 0
IS = 0
DS = 0
rcc = 0
isc = 100

start_time = time.time()
pe = {}
e = {}
idx = 0

speed_limit = 50
red_light_violations = False

set_cam_to_EGO(ego)
camera.listen(image_to_emb) #changes emb, image_1 globally

collision_sensor = CollisionSensor(ego)

init_pos = ego.get_transform().location
distance = 0.0
fin_pos = init_pos

while emb is None:
    world.tick()

if prev_emb:
    for i in prev_emb.keys():
        pe[i] = prev_emb[i]
for i in emb.keys():
    e[i] = emb[i]

while True:
    world.wait_for_tick()
    e = emb['pixel_values']
    # print(f"unique emb['pixel_values]: {e}")
    if prev_emb:
        pe = prev_emb['pixel_values']
        # print(f"unique prev_emb['pixel_values]: {pe}")
        if not torch.equal(e,pe):
            idx = idx + 1

            outputs = model_output_generation(e, driver_model)
            waypoints = outputs.detach().numpy()

            scene_h, scene_w = process_image(image_1).size
            ego_x_mid = scene_h/2
            ego_y_mid = scene_w/2

            ego_x_position = ego_x_mid
            ego_y_position = 25

            waypoints = normalize_wp(waypoints, scene_w, scene_h)

            intended_y_mid = np.mean([point[0] for point in waypoints])
            intended_x_mid = np.mean([point[1] for point in waypoints])

            curr_pos = ego.get_transform().location
            dx = curr_pos.x - fin_pos.x
            dy = curr_pos.y - fin_pos.y
            dz = curr_pos.z - fin_pos.z
            d = math.sqrt((dx ** 2) + (dy ** 2) + (dz ** 2))
            distance = distance + d

            evaluate_RC(intended_x_mid, ego_x_position, intended_y_mid, ego_y_position)
            evaluate_IS(speed_limit, red_light_violations, collision_sensor)

            throttle, brake = TandB(ego_y_mid, intended_y_mid, ego_y_position)

            velocity = ego.get_velocity()
            speed = math.sqrt((velocity.x**2 + velocity.y**2 + velocity.z**2)) / 5
            s_power = (1 - brake) * (1 - speed)

            steer = steering(ego_x_position, intended_x_mid, s_power)

            # print(outputs, waypoints, new_wp)
            print(f"\rcurrently at: ({ego_x_position:.1f}, {ego_y_position:.1f}), mid: ({ego_x_mid:.1f}, {ego_y_mid:.1f}), going towards: ({intended_x_mid:.1f}, {intended_y_mid:.1f}), with b:{brake}, t:{throttle:.1f}, s:{steer:.1f} and ISc:{isc}, RCc: {rcc}      ", end = "")

            control = carla.VehicleControl()
            waypoints = outputs.detach().numpy()
            control.steer = steer
            control.throttle = throttle
            control.brake = brake
            ego.apply_control(control)
            set_cam_to_EGO(ego)
            
    if(time.time() >= start_time + 60):
        break
    prev_emb = emb

if distance > 0:
    RC = rcc * 100 / distance
    IS = isc * 100 / distance
    evaluate_DS()

print(f"\nRC is: {RC:.2f}, IS is: {IS:.2f}, DS is: {DS:.2f}, ")

started
currently at: (960.0, 25.0), mid: (960.0, 540.0), going towards: (402.5, 570.6), with b:0, t:0.6, s:-0.3 and ISc:100, RCc: 35        
RC is: 5.30, IS is: 15.13, DS is: 0.00, 


In [33]:
camera.stop()
remove_all_actors()

ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

set_cam_to_EGO(ego, 65)
print("stopped")

previous ones destroyed
stopped


currently being worked upon

In [89]:
print("started")

emb = None
prev_emb = None
image_1 = None

RC = 0
IS = 0
DS = 0
rcc = 0
isc = 100

start_time = time.time()
pe = {}
e = {}
idx = 0

speed_limit = 50
red_light_violations = False

set_cam_to_EGO(ego)
camera.listen(image_to_emb) #changes emb, image_1 globally

collision_sensor = CollisionSensor(ego)

init_pos = ego.get_transform().location
distance = 0.0
fin_pos = init_pos

while emb is None:
    world.tick()

if prev_emb:
    for i in prev_emb.keys():
        pe[i] = prev_emb[i]
for i in emb.keys():
    e[i] = emb[i]

while True:
    world.wait_for_tick()
    e = emb['pixel_values']
    # print(f"unique emb['pixel_values]: {e}")
    if prev_emb:
        pe = prev_emb['pixel_values']
        # print(f"unique prev_emb['pixel_values]: {pe}")
        if not torch.equal(e,pe):
            idx = idx + 1

            outputs = model_output_generation(e, driver_model)
            waypoints = outputs.detach().numpy()

            scene_x, scene_y = process_image(image_1).size
            # print(scene_x, scene_y)
            ego_x_mid = scene_x/2
            ego_y_mid = scene_y/2

            ego_x_position = ego_x_mid
            ego_y_position = 75

            maxiy = -float('inf')
            maxix = -float('inf')
            for i in waypoints:
                if maxix < i[0]:
                    maxix = i[0]
                if maxiy < i[1]:
                    maxiy = i[1]

            sumx = 0
            sumy = 0
            for i in waypoints:
                i[0] = i[0] * scene_x / maxix
                i[1] = i[1] * scene_y / maxiy
                sumx = sumx + i[0]
                sumy = sumy + i[1]

            # waypoints = normalize_wp(waypoints, scene_x, scene_y)

            intended_x_mid = sumx / 10
            intended_y_mid = sumy / 10
            intended_y_mid = scene_y - intended_y_mid
                
            # print(waypoints)

            # print(intended_x_mid, intended_y_mid)

            # break

            curr_pos = ego.get_transform().location
            dx = curr_pos.x - fin_pos.x
            dy = curr_pos.y - fin_pos.y
            dz = curr_pos.z - fin_pos.z
            d = math.sqrt((dx ** 2) + (dy ** 2) + (dz ** 2))
            distance = distance + d

            evaluate_RC(intended_x_mid, ego_x_position, intended_y_mid, ego_y_position)
            evaluate_IS(speed_limit, red_light_violations, collision_sensor)

            throttle, brake = TandB(ego_y_mid, intended_y_mid, ego_y_position)

            velocity = ego.get_velocity()
            speed = math.sqrt((velocity.x**2 + velocity.y**2 + velocity.z**2)) / 5
            s_power = (1 - brake) * (1 - speed)

            steer = steering(ego_x_position, intended_x_mid, s_power)

            # print(outputs, waypoints, new_wp)
            print(f"\rcurrently at: ({ego_x_position:.1f}, {ego_y_position:.1f}), mid: ({ego_x_mid:.1f}, {ego_y_mid:.1f}), going towards: ({intended_x_mid:.1f}, {intended_y_mid:.1f}), with b:{brake}, t:{throttle:.1f}, s:{steer:.1f}      ", end = "")

            control = carla.VehicleControl()
            waypoints = outputs.detach().numpy()
            control.steer = steer
            control.throttle = throttle
            control.brake = brake
            ego.apply_control(control)
            set_cam_to_EGO(ego)
            
    if(time.time() >= start_time + 400):
        break
    prev_emb = emb
if(distance > 0):
    RC = rcc * 100 / distance
    IS = isc * 100 / distance
evaluate_DS()

print(f"\nRC is: {RC:.2f}, DS is: {DS:.2f}, IS is: {IS:.2f}")

started
currently at: (960.0, 75.0), mid: (960.0, 540.0), going towards: (1638.2, 140.5), with b:0, t:0.3, s:0.2         

KeyboardInterrupt: 

In [90]:
camera.stop()
remove_all_actors()

ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

set_cam_to_EGO(ego, 65)
print("stopped")

previous ones destroyed
stopped
